# ソフトマックス関数について

## 手書き数字認識の場合のSoftmaxレイヤの出力
![softmax_function](https://files.slack.com/files-tmb/T0FNB0BP1-F4LMCKYPP-07885f2b72/____________________________2017-03-20_15.39.34_1024.png)

Softmaxレイヤは入力された値を正規化（出力のわが1になるように変形）して出力します。


ニューラルネットワークで行う処理には、推論(inference)と学習のフェーズがある。
推論では、通常Softmaxレイヤは使用しない。最後のAffineレイヤの出力（スコア）を認識結果として用いる。
推論で答えと一つだけ出す場合は、スコア裁断値だけに興味があるため、Softmaxレイヤは必要ない。

## Softmax with Lossレイヤの実装
Softmaxと損失関数である交互エントロピー誤差（cross entropy error)も含めたレイヤ
![Softmax-with-loss-layer](https://files.slack.com/files-pri/T0FNB0BP1-F4LJWSWCS/____________________________2017-03-20_16.05.11.png)

計算グラフではソフトマックス関数はSoftmaxレイヤとして、交差エントロピー誤差はCross Extropy Errorレイヤとして表記する。

1. Softmaxレイヤは入力である$(a_1, a_2, a_3)$を正規化して、$(y_1, y_2, y_3)$を出力する。
2. Cross Entropy Errorレイヤは Softmaxの出力$(y_1, y_2, y_3)$と教師ラベルの$(t_1, t_2, t_3)$を受取、それらのデータから損失 Lを出力する


## 逆伝播の結果について
Softmaxレイヤからの逆伝播$(y_1 - t_1, y_2 - t_2, y_3 - t_3 )$ はSoftmaxレイヤの出力と教師ラベルの差分になる。
<strong>ニューラルネットワークの逆伝播では、この差分である誤差が前レイヤへ伝わっていく。</strong>

ニューラルネットワークの目的は、ニューラルネットワークの出力（Softmaxの出力）を教師ラベルに近づけるように、重みパラメータを調整すること。
そのため、ニューラルネットワークの出力と教師ラベルとの誤差を効率よく前レイヤに伝える必要がある。
Softmaxレイヤからの逆伝播はまさに、出力と講師ラベルの差である。

交差エントロピー誤差という関数は、ソフトマックス関数用の損失関数として設計されたものなので、逆伝播の値がキレイな教師ラベルとの誤差となる。
回帰問題では、出力層に「恒等関数」を用い、損失関数として「2乗和誤差」を用いるのも同様の理由。

#### 例1
- 教師ラベルが(0,1,0)
- SoftMaxレイヤの出力が(0.3, 0.2, 0.5)
-  正解ラベルに対する確率は0.2 (20% = 1 × 0.2)

この時点のニューラルネットワークは正しい認識ができていない。
Softmaxレイヤからの逆伝播は、(0.3, -0.8, 0.5)という大きな誤差を伝播することになる。

#### 例2
- 教師ラベルが(0, 1, 0)
- Softmaxレイヤの出力が(0.01, 0.99, 0)

このニューラルネットワークはかなり正確に認識している。
この場合のSoftmaxレイヤからの逆伝播は(0.01, -0.01, 0)と小さな誤差になるた、えSoftmaxレイヤより前にあるレイヤが学習する内容も小さくなる。


In [5]:
import numpy as np

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 損失
        self.y = None # softmaxの出力
        self.t = None # 教師データ(one-hot vector)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(y)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size  = self.t.shape[0]
        # 伝播する値をバッチの個数（batch_size）で割ることで、データ1個あたりの誤差が前レイヤへ伝播する
        dx = (self.y - self,t) / batch_size 

        return dx

    def softmax(a):
        c = np.max(a)
        exp_a = np.exp(a - c) # オーバーフロー対策
        sum_exp_a = np.sum(exp_a)
        y = exp_a / sum_exp_a

        return y

    def cross_entropy_error(y, t):
        delta = 1e-7
        return -np.sum(t * np.log(y + delta))